In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import os
os.chdir('/content/drive/My Drive/Pose Estimation/orvee/Pretrained openpose')

Mounted at /content/drive


In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [ ]:
!pip install tensorflow==2.2

     |████████████████████████████████| 516.2MB 23kB/s 
     |████████████████████████████████| 3.0MB 45.0MB/s 
     |████████████████████████████████| 460kB 41.6MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
from __future__ import division
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import Model, Sequential, utils
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import load_model

import pandas as pd 
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from random import shuffle
import os
from keras.models import Model, Sequential, load_model
from keras import layers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D, ZeroPadding2D, concatenate, Input, Permute, Concatenate, BatchNormalization
from keras import applications
#from keras.utils import plot_model, to_categorical
from keras.applications import VGG16, ResNet50
from keras.optimizers import RMSprop,Adam  
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import pickle
import random
import cv2

In [ ]:
# DATA LOADER
data_dict = dict()
data_dict_test = dict()
validation_dict = dict()
test_dict = dict()

data_dict['X'] = np.load('bangla_dataset/np_files/imglist_t.npy', allow_pickle=True) #DATASET.train_folder + '/images.npy'
data_dict['X2'] = np.load('bangla_dataset/np_files/poselist_t.npy', allow_pickle=True)
data_dict['Y'] = np.load('bangla_dataset/np_files/labellist_t.npy', allow_pickle=True)

data_dict_test['X'] = np.load('bangla_dataset/np_files/imglist_v.npy', allow_pickle=True) #DATASET.train_folder + '/images.npy'
data_dict_test['X2'] = np.load('bangla_dataset/np_files/poselist_v.npy', allow_pickle=True)
data_dict_test['Y'] = np.load('bangla_dataset/np_files/labellist_v.npy', allow_pickle=True)



### Main Model

In [ ]:
image_network = Input(shape = [64, 64, 1], name = "input_1")
x=Convolution2D(64, (3, 3), padding = 'same', input_shape = (64, 64))(image_network)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(64, (3, 3), padding = 'same')(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=Dropout(0.2)(x)

x=Convolution2D(128, (3, 3), padding = 'same')(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(128, (3, 3), padding = 'same')(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=Dropout(0.2)(x)

x=Convolution2D(256, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(256, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(256, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=Dropout(0.2)(x)

x=Convolution2D(512, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(512, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(512, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=Dropout(0.3)(x)

x = Flatten()(x)
x = Dense(1024, activation = 'relu')(x)
x=BatchNormalization()(x)
x=Dropout(0.3)(x)
w = Dense(512, activation = 'relu')(x)
w=Dropout(0.3)(w)
pose_network = Input(shape = [21, 2], name = "input_2")

x2 = Flatten()(pose_network)
x2 = Dense(1024, activation = 'relu')(x2)
x2=Dropout(0.3)(x2)
z = Dense(512, activation = 'relu')(x2)
z=Dropout(0.3)(z)


merged_network = concatenate([w, z], axis = 1)
merged_network = Dense(512, activation = 'elu')(merged_network)
merged_network = Dense(256, activation = 'elu')(merged_network)
merged_network=BatchNormalization()(merged_network)
merged_network = Dense(38, activation = 'softmax')(merged_network)

model = Model(inputs = [image_network, pose_network], outputs = merged_network)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 64)   256         activation[0][0]                 
______________________________________________________________________________________________

In [ ]:
# model = load_model("final_weights/64_MainModelepoch30.h5")

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.00020000000949949026),
             metrics=["accuracy"])

In [ ]:
checkpoint = ModelCheckpoint(
                             'final_weights/64_MainModelepoch20_30.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True, 
                             verbose=1)

earlystop = EarlyStopping(                   
                          monitor='val_loss',
                          min_delta=0,
                          restore_best_weights=True,
                          patience=20,
                          verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.000001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

In [ ]:
H = model.fit([data_dict['X'], data_dict['X2']],
	data_dict['Y'],
	validation_data=([data_dict_test['X'], data_dict_test['X2']],
	data_dict_test['Y']),
	epochs=30,
  callbacks=callbacks,
	verbose=1)

Epoch 1/30
312/312 [==============================] - ETA: 0s - loss: 2.3383 - accuracy: 0.3296
Epoch 00001: val_loss improved from inf to 5.47874, saving model to final_weights/64_MainModelepoch30.h5
312/312 [==============================] - 1512s 5s/step - loss: 2.3383 - accuracy: 0.3296 - val_loss: 5.4787 - val_accuracy: 0.1633 - lr: 0.0010
Epoch 2/30
312/312 [==============================] - ETA: 0s - loss: 1.1813 - accuracy: 0.6268
Epoch 00002: val_loss improved from 5.47874 to 0.97384, saving model to final_weights/64_MainModelepoch30.h5
312/312 [==============================] - 1518s 5s/step - loss: 1.1813 - accuracy: 0.6268 - val_loss: 0.9738 - val_accuracy: 0.6878 - lr: 0.0010
Epoch 3/30
312/312 [==============================] - ETA: 0s - loss: 0.9100 - accuracy: 0.7124
Epoch 00003: val_loss did not improve from 0.97384
312/312 [==============================] - 1508s 5s/step - loss: 0.9100 - accuracy: 0.7124 - val_loss: 1.0742 - val_accuracy: 0.6815 - lr: 0.0010
Epoch 4/3

In [ ]:
H = model.fit([data_dict['X'], data_dict['X2']],
	data_dict['Y'],
	validation_data=([data_dict_test['X'], data_dict_test['X2']],
	data_dict_test['Y']),
	epochs=10,
  callbacks=callbacks,
	verbose=1)

Epoch 1/10
312/312 [==============================] - ETA: 0s - loss: 0.0956 - accuracy: 0.9696
Epoch 00001: val_loss improved from inf to 0.20335, saving model to final_weights/64_MainModelepoch20_30.h5
312/312 [==============================] - 1660s 5s/step - loss: 0.0956 - accuracy: 0.9696 - val_loss: 0.2034 - val_accuracy: 0.9392 - lr: 2.0000e-04
Epoch 2/10
312/312 [==============================] - ETA: 0s - loss: 0.0757 - accuracy: 0.9755
Epoch 00002: val_loss improved from 0.20335 to 0.19003, saving model to final_weights/64_MainModelepoch20_30.h5
312/312 [==============================] - 1663s 5s/step - loss: 0.0757 - accuracy: 0.9755 - val_loss: 0.1900 - val_accuracy: 0.9474 - lr: 2.0000e-04
Epoch 3/10
312/312 [==============================] - ETA: 0s - loss: 0.0668 - accuracy: 0.9795
Epoch 00003: val_loss did not improve from 0.19003
312/312 [==============================] - 1666s 5s/step - loss: 0.0668 - accuracy: 0.9795 - val_loss: 0.2043 - val_accuracy: 0.9428 - lr: 2.

In [ ]:
model = model.save("final_weights/64_MainModelepoch30th.h5")

In [ ]:
score = model.evaluate([data_dict_evaluate['X'], data_dict_evaluate['X2']],
	data_dict_evaluate['Y'])

48/48 [==============================] - 59s 1s/step - loss: 0.3589 - accuracy: 0.9151


In [ ]:
H = model.fit([data_dict['X'], data_dict['X2']],
	data_dict['Y'],
	validation_data=([data_dict_test['X'], data_dict_test['X2']],
	data_dict_test['Y']),
	epochs=30,
  callbacks=callbacks,
	verbose=1)

Epoch 1/30
312/312 [==============================] - ETA: 0s - loss: 2.1492 - accuracy: 0.3675
Epoch 00001: val_loss improved from inf to 2.89629, saving model to trained_weights/VGG + 512 DENSE ADDITION RELU_epoch30.h5
312/312 [==============================] - 1681s 5s/step - loss: 2.1492 - accuracy: 0.3675 - val_loss: 2.8963 - val_accuracy: 0.4047 - lr: 0.0010
Epoch 2/30
312/312 [==============================] - ETA: 0s - loss: 1.1629 - accuracy: 0.6305
Epoch 00002: val_loss improved from 2.89629 to 1.21503, saving model to trained_weights/VGG + 512 DENSE ADDITION RELU_epoch30.h5
312/312 [==============================] - 1683s 5s/step - loss: 1.1629 - accuracy: 0.6305 - val_loss: 1.2150 - val_accuracy: 0.6316 - lr: 0.0010
Epoch 3/30
312/312 [==============================] - ETA: 0s - loss: 0.9305 - accuracy: 0.7036
Epoch 00003: val_loss improved from 1.21503 to 1.06417, saving model to trained_weights/VGG + 512 DENSE ADDITION RELU_epoch30.h5
312/312 [============================

In [ ]:
H = model.fit([data_dict['X'], data_dict['X2']],
	data_dict['Y'],
	validation_data=([data_dict_test['X'], data_dict_test['X2']],
	data_dict_test['Y']),
	epochs=12,
  callbacks=callbacks,
	verbose=1)

Epoch 1/12
312/312 [==============================] - ETA: 0s - loss: 0.0961 - accuracy: 0.9719
Epoch 00001: val_loss improved from inf to 0.18376, saving model to trained_weights/VGG + 512 DENSE ADDITION RELU_epoch18-30.h5
312/312 [==============================] - 1659s 5s/step - loss: 0.0961 - accuracy: 0.9719 - val_loss: 0.1838 - val_accuracy: 0.9428 - lr: 1.0000e-04
Epoch 2/12
312/312 [==============================] - ETA: 0s - loss: 0.0858 - accuracy: 0.9740
Epoch 00002: val_loss did not improve from 0.18376
312/312 [==============================] - 1664s 5s/step - loss: 0.0858 - accuracy: 0.9740 - val_loss: 0.1968 - val_accuracy: 0.9474 - lr: 1.0000e-04
Epoch 3/12
312/312 [==============================] - ETA: 0s - loss: 0.0828 - accuracy: 0.9755
Epoch 00003: val_loss did not improve from 0.18376
312/312 [==============================] - 1666s 5s/step - loss: 0.0828 - accuracy: 0.9755 - val_loss: 0.2002 - val_accuracy: 0.9410 - lr: 1.0000e-04
Epoch 4/12
312/312 [============

In [ ]:
model = load_model("final_weights/64_MainModelepoch30th.h5")

In [ ]:
data_dict_evaluate = dict()
data_dict_evaluate['X'] = np.load('imglist_test.npy', allow_pickle=True)
data_dict_evaluate['X2'] = np.load('poselist_test.npy', allow_pickle=True)
data_dict_evaluate['Y'] = np.load('labellist_test.npy', allow_pickle=True)

In [ ]:
score = model.evaluate([data_dict_evaluate['X'], data_dict_evaluate['X2']],
	data_dict_evaluate['Y'])

48/48 [==============================] - 60s 1s/step - loss: 0.3589 - accuracy: 0.9151
